In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import pandas as pd 
import numpy as np
import os

#import data 
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/raw_data/"
os.getcwd()

Mounted at /content/drive
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [ ]:
from os import listdir
FILE_LIST = listdir(DATA_PATH)
print(FILE_LIST)

df_dic={} # create a dictionary with names of datasets as keys and datasets as values
for i, e in enumerate(FILE_LIST):
  df_dic[str(FILE_LIST[i].split(".csv")[0])]= pd.read_csv("raw_data/" + e)

# test
#df_dic['diaries_transactions_all'].columns

['con_prod_hh_over_trx_yr_mo_usd.csv', 'con_pur_hh_by_trx_yr_mo_usd.csv', 'demo_hh.csv', 'demo_ind.csv', 'diaries_goingson_hh.csv', 'diaries_wellbeing_ind.csv', 'edu_hh.csv', 'edu_ind.csv', 'housing_conditions_ownership.csv', 'inc_hh_by_trx_yr_mo_usd_incRR.csv', 'inc_ind_by_trx_yr_mo_usd_incRR.csv', 'pov_hh_by_trx_yr_mo_usd_incRR.csv', 'rem_hh_by_trx_yr_mo_usd.csv', 'pov_hh_by_trx_yr_mo_usd_excRR.csv', 'diaries_transactions_all.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Start merging
#### first we merge datasets with same keys together, then we merge the bigger (already merged) datasets together

In [ ]:
from functools import reduce

dfs = [df_dic['diaries_transactions_all'], df_dic['demo_hh'] , df_dic['con_prod_hh_over_trx_yr_mo_usd'], df_dic['edu_hh']]

result_hhids = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = 'hh_ids', left_on = 'hh_ids', 
                                              how='outer'), 
                  dfs)

In [ ]:
result_hhids.head()

In [ ]:
dfs = [df_dic['con_pur_hh_by_trx_yr_mo_usd'], df_dic['inc_hh_by_trx_yr_mo_usd_incRR'], df_dic['pov_hh_by_trx_yr_mo_usd_excRR'], df_dic['pov_hh_by_trx_yr_mo_usd_incRR'], df_dic['rem_hh_by_trx_yr_mo_usd']]

result_hhids_trxyrmo = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer'),
                  dfs)

In [ ]:
result_hhids_trxyrmo.head()

In [ ]:
dfs = [df_dic['diaries_wellbeing_ind'], df_dic['diaries_goingson_hh']]

result_hhids_intdate = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer'),
                  dfs)


In [ ]:
result_hhids_intdate.head()

In [ ]:
dfs = [df_dic['demo_ind'], df_dic['edu_ind']]

result_hhids_inds = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids'], 
                                              how= 'outer'),
                  dfs)

result_hhids_inds['m_ids'] = result_hhids_inds['m_ids'].astype(str) # convert to string for merging purposes

In [ ]:
result_hhids_inds.head() 

In [ ]:
# final part of merging

official_1 = pd.merge(result_hhids, result_hhids_trxyrmo, right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer')



In [ ]:
official_2 = pd.merge(official_1, result_hhids_intdate, right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer')

In [ ]:
official= pd.merge(official_2, result_hhids_inds, right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids_trx'], 
                                              how= 'outer')


In [ ]:
official.head()

In [ ]:
official[official['hh_ids'] == 'KELDK01'] # check 

final comment on merging --> DID NOT MANAGE TO MERGE inc_ind_by_trx_yr_mo_usd_incRR and housing_conditions --> remember to include limitations of this choice in our report!

In [ ]:
official.replace('.', np.NaN) # replace NAs consistently along the whole dataset
official.replace(' ', np.NaN)

In [ ]:
empty_cols = [col for col in official.columns if official[col].isnull().all()] # checking for empty columns 

print(empty_cols) # no empty columns

[]


In [ ]:
# drop unimportant cols 

official = official.drop(columns = ['trx_yr_mo','first_trx_date_hh', 'last_trx_date_hh', 'tot_hh_daysofobs', 'tot_hh_monthsofobs', 'interview_designation', 'int_date', 'int_month', 'int_year', 'int_yr_mo', 'first_int_date', 'unique_accnts', 'unique_hm_owner', 'tot_acc_daysofobs', 'tot_acc_monthsofobs', 'trx_stdtime_days_hh', 'trx_stdtime_mnths_hh', 'trx_stdtime_days_acc', 'trx_stdtime_mnths_acc', 'trx_family_desc', 'trx_class_desc', 'trx_type_desc', 'trx_mode_code', 'trx_place_incommunity', 'trx_distance_km', 'trx_value_kes', 'trx_inkind_units', 'trx_inkind_value_kes'], axis=1)


In [ ]:
# drop trx_class_code column

official = official.drop(columns = ['trx_class_code'], axis = 1) # motivation: conflict with target prediction, see discussion in the report

In [ ]:
# drop cols with more than 80% of NaNs

na_perc = official[official.columns[official.isnull().any()]].isnull().sum() * 100 / official.shape[0]
#print(na_perc)

nas_list = []
for i in na_perc.keys():
  if na_perc[i] > 80.0:
    nas_list.append(i)

official = official.drop(columns = nas_list) # DROP MOSTLY EMPTY COLUMNS

official.shape

(735429, 165)

In [ ]:
perc = 80.0
min_count =  int(((100-perc)/100)*official.shape[1] + 1) # DROP MOSTLY EMPTY ROWS
official = official.dropna(axis=0, 
                    thresh=min_count) 

official.shape

(734904, 165)

In [12]:
%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/MLProject_KenyaFinancial/clean_data/official.csv"
os.getcwd()

official = pd.read_csv(DATA_PATH)

/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,150) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Drop NaN values for the target value

In [13]:
official = official[official['trx_mode_desc'].notna()] # drop na rows for trx_mode_desc

In [14]:
for column in official.columns: # convert them into the same object
  if official[column].dtype == "object":
    official[column] = official[column].astype("|S")

Treat important variables that have inconsistent values: 'trx_outlet' and 'trx_purpose'

In [50]:
Tol = []
for tol in official["trx_outlet"]:
    Tol.append(tol.decode('UTF-8'))

In [28]:
#!pip install stop-words

from stop_words import get_stop_words

In [51]:
sw = get_stop_words('english')
sw.extend(['large', 'small', 'kenya', 'kenyan', 'place', 'person'])
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

Tol1 = []
for val in Tol:
    val1 = ''.join(filter(whitelist.__contains__, val))
    val1 = val1.lower()
    val1 = ' '.join([word for word in val1.split() if word not in sw])
    Tol1.append(val1)

In [70]:
singlewords= []
for sentence in Tol1:
  firstword = sentence.split(sep=None, maxsplit=1)
  try:
    singlewords.append(firstword[0])
  except:
    if type(firstword) != 'list':
          singlewords.append(firstword)
    else:
      singlewords.append('')



In [71]:
official["trx_outlet1"] = singlewords

In [73]:
official["trx_outlet1"].astype(str).nunique()


147

In [75]:
Tol = []
for tol in official["trx_prx_purpose"]:
    Tol.append(tol.decode('UTF-8'))
###############
sw = get_stop_words('english')
sw.extend(['large', 'small', 'kenya', 'kenyan', 'place', 'person'])
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
##############
Tol1 = []
for val in Tol:
    val1 = ''.join(filter(whitelist.__contains__, val))
    val1 = val1.lower()
    val1 = ' '.join([word for word in val1.split() if word not in sw])
    Tol1.append(val1)
##############
singlewords= []
for sentence in Tol1:
  firstword = sentence.split(sep=None, maxsplit=1)
  try:
    singlewords.append(firstword[0])
  except:
    if type(firstword) != 'list':
          singlewords.append(firstword)
    else:
      singlewords.append('')
#############
official["trx_purpose1"] = singlewords


In [76]:
official["trx_purpose1"].astype(str).nunique()

55

Drop old columns and add new ones

In [80]:
official = official.drop(['trx_outlet', 'trx_prx_purpose'], axis = 1)

Merged dataframe is ready to be split into train and test!

In [81]:
os.getcwd()

'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [83]:
os.makedirs('clean_data', exist_ok=True)  # make dataset available as csv for machine learning preprocessing
compression_opts = dict(method='zip',
                        archive_name='official.csv')
official.to_csv('clean_data/official.zip', index=False,
          compression=compression_opts) 